In [1]:
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as utils_data

In [2]:
from google.colab import drive
drive.mount('/content/drive')

from model import HawkesTransformer
from train import train, run_epoch
from dataset_wrapper import LobDataset, prepare_datasets
from utils import fix_seed

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
TRAIN_MODEL = True # set to False if you wish to use weights
WEIGHT_NAME = None # change if you wish to upload existing weights (without pth)
GLOBAL_DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
GLOBAL_SEED = 42
DATA_DIR = "/content/drive/MyDrive/LOB datasets/"

In [4]:
torch.cuda.empty_cache()
!nvidia-smi

Wed Dec 16 14:23:21 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.45.01    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0    46W / 300W |     10MiB / 16130MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
eth_data = np.load(DATA_DIR + "ETH.npy")

train_size, val_size = 0.6, 0.2
train_part, val_part = int(train_size * len(eth_data)), int(val_size * len(eth_data))
train_dataset, val_dataset, test_dataset = LobDataset(eth_data[:train_part], slice_inputs=False),\
                                           LobDataset(eth_data[train_part:train_part + val_part], slice_inputs=False),\
                                           LobDataset(eth_data[train_part + val_part:], slice_inputs=False)

train_loader, val_loader, test_loader = utils_data.DataLoader(train_dataset, batch_size=5, shuffle=True),\
                                        utils_data.DataLoader(val_dataset, batch_size=5, shuffle=False),\
                                        utils_data.DataLoader(val_dataset, batch_size=5, shuffle=False)

In [6]:
from itertools import product

params = {
    'd_model' : {64, 256, 512},
    'n_heads' : {2, 4},
    'dim_feedforward' : {512, 1024}
}

best_loss = float('inf')
best_params = {}
for param in product(*params.values()):
    fix_seed(GLOBAL_SEED)
    model = HawkesTransformer(2, GLOBAL_DEVICE, n_layers=4, dropout=0.1, activation='relu', **dict(zip(params, param))).to(GLOBAL_DEVICE)

    optimizer = optim.Adam(model.parameters(), lr=1e-4, eps=1e-5, weight_decay=1e-4)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.5, threshold=0.001, patience=10, verbose=True)
    n_epochs  = 30

    train_history, val_history = train(model, n_epochs, optimizer, train_loader, val_loader, scheduler, GLOBAL_DEVICE, verbose=True, freq=10,\
                                       early_stopping=False, patience=15, checkpoint=True, cp_name='model_eth')
    ll_loss, tp_loss, ec_loss, accuracy = run_epoch(model, val_loader, GLOBAL_DEVICE)
    val_loss = -ll_loss + tp_loss + ec_loss
    if val_loss < best_loss:
        best_loss = val_loss
        best_params = param

[ Epoch 10 ]
(Training)     log-likelihood: -3.8115910665133836, RMSE: 17.563193115976368, CE: 0.6025968426505763, accuracy: 0.7140302041608806
(Validation)   log-likelihood: -4.078514905753968, RMSE: 21.262598958049523, CE: 0.6070271966895712, accuracy: 0.7052922402705664
Time elapsed: 69.27 s
[ Epoch 20 ]
(Training)     log-likelihood: -3.804102891366693, RMSE: 15.731807332554105, CE: 0.6025736263580311, accuracy: 0.7140291489568252
(Validation)   log-likelihood: -4.073759176587302, RMSE: 18.954971752222853, CE: 0.6073017673798012, accuracy: 0.7052922402705664
Time elapsed: 138.52 s
[ Epoch 30 ]
(Training)     log-likelihood: -3.797431189428402, RMSE: 14.992630165647931, CE: 0.6010018450449003, accuracy: 0.7140333697730468
(Validation)   log-likelihood: -4.061077132936508, RMSE: 17.724484938760195, CE: 0.6067869225257626, accuracy: 0.7052922402705664
Time elapsed: 207.76 s
[ Epoch 10 ]
(Training)     log-likelihood: -3.8182498341491957, RMSE: 17.44105237263555, CE: 0.6022028777339513

In [7]:
best_params

(256, 4, 1024)